In [30]:
import javalang
import pandas as pd


In [2]:
f = open('complexity_data/origin/1104.java')
source = f.read()
f.close()

In [28]:
f = open('complexity_data/origin/1105.java')
source = f.read()
f.close()

In [31]:
tree=javalang.parse.parse(source)

In [9]:
tree==tree2

False

In [69]:

for _,node in tree:
    if type(node)== javalang.tree.ConstructorDeclaration:
        print(node)
        sample=node
        

ConstructorDeclaration(annotations=[], body=[StatementExpression(expression=Assignment(expressionl=MemberReference(member=reader, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), type==, value=ClassCreator(arguments=[ClassCreator(arguments=[MemberReference(member=stream, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=InputStreamReader, sub_type=None))], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=BufferedReader, sub_type=None))), label=None), StatementExpression(expression=Assignment(expressionl=MemberReference(member=st, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), type==, value=Literal(postfix_operators=[], prefix_op

In [135]:
sample.types

AttributeError: 'ConstructorDeclaration' object has no attribute 'types'

In [215]:
a=' tos'

a.lstrip()

'tos'

In [106]:
['modifiers',
 'annotations',
 'documentation',
 'type_parameters',
 'name',
 'parameters',
 'throws',
 'body']

['modifiers',
 'annotations',
 'documentation',
 'type_parameters',
 'name',
 'parameters',
 'throws',
 'body']

In [34]:
root=tree.__iter__().__next__()[1]

In [65]:
func_dict={"ClassCreator":ClassCreator2Code,
"Literal":Literal2Code,
    'BinaryOperation':BinaryOperation2Code,
    'MemberReference':MemberReference2Code,
    'BlockStatement':BlockStatement2Code,
    'LocalVariableDeclaration':LocalVariableDeclaration2Code,
    'VariableDeclarator':VariableDeclarator2Code,
    'ArrayCreator': ArrayCreator2Code,
    'StatementExpression':StatementExpression2Code,
    'ArraySelector':ArraySelector2Code,
    'MethodInvocation':MethodInvocation2Code,
    'Assignment':Assignment2Code,
    'IfStatement':IfStatement2Code,
    'ForStatement':ForStatement2Code,
    'MethodDeclaration':MethodDeclaration2Code,
    'Cast':Cast2Code,
    'ReturnStatement':ReturnStatement2Code,
    'ClassDeclaration':ClassDeclaration2Code,
    'FieldDeclaration':FieldDeclaration2Code,
    'NoneType':Nonetype,
    'ConstructorDeclaration':ConstructorDeclaration2Code,
    'WhileStatement':WhileStatement2Code
}

In [56]:

root.types[0].body[1].body[4].body

BlockStatement(label=None, statements=[LocalVariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=MethodInvocation(arguments=[], member=nextInt, postfix_operators=[], prefix_operators=[], qualifier=sc, selectors=[], type_arguments=None), name=x)], modifiers=set(), type=BasicType(dimensions=[], name=int)), IfStatement(condition=BinaryOperation(operandl=MemberReference(member=x, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), operandr=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=4), operator=<=), else_statement=None, label=None, then_statement=StatementExpression(expression=MemberReference(member=freq, postfix_operators=['++'], prefix_operators=[], qualifier=, selectors=[ArraySelector(index=MemberReference(member=x, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]))]), label=None))])

In [67]:
tree2=javalang.parse.parse(AST2Code(root))

JavaSyntaxError: 

In [2]:
def AST2Code(Root):
    code=''
    for import_item in Root.imports:
        code+= 'import '+ import_item.path+ ('.*' if import_item.wildcard else '') + ';\n'
    childs=root.types
    for child in root.types:
        code+=func_dict[type(child).__name__](child)
    return code

In [3]:
def Nonetype(node):
    return ''

In [4]:
def FieldDeclaration2Code(node):
    code_snippet=''
    code_snippet+=Type2Code(node.type)
    for modifier in node.modifiers:
        code_snippet+= " "+modifier
    for declarator in node.declarators:
        code_snippet+= " "+func_dict[type(declarator).__name__](declarator)

    return code_snippet

In [64]:
def WhileStatement2Code(node):
    code_snippet='while('+func_dict[type(node.condition).__name__](node.condition)+')'
    code_snippet+=func_dict[type(node.body).__name__](node.body)
    return code_snippet

In [6]:
def ConstructorDeclaration2Code(node):
    code_snippet=''
    for modifier in node.modifiers:
        code_snippet+=' '+modifier
    code_snippet+=' '+ node.name
    code_snippet=code_snippet.lstrip()
    paramter=''
    for pram in node.parameters:
        paramter+=pram.type.name + ('[]'*len(pram.type.dimensions)) +' '+ pram.name +','
    code_snippet+='('+paramter.rstrip(',')+'){\n'
    for child in node.body:
        code_snippet+= func_dict[type(child).__name__](child)
    return code_snippet+'}\n'

In [7]:
def ClassDeclaration2Code(node):
    code_snippet=''
    for modifier in node.modifiers:
        code_snippet+=modifier+' ' 
    code_snippet+=' class '+node.name +'{\n'
    
    for class_item in node.body:
        code_snippet+=func_dict[type(class_item).__name__](class_item)

    return code_snippet+"}"

In [8]:
def MethodDeclaration2Code(node):
    code_snippet=''
    for modifier in node.modifiers:
        code_snippet+=modifier+' ' 
    code_snippet+=(Type2Code(node.return_type) if node.return_type else 'void')+' '+node.name+'('
    paramter=''
    for pram in node.parameters:
        paramter+=pram.type.name + ('[]'*len(pram.type.dimensions)) +' '+ pram.name +','

    code_snippet+= paramter.rstrip(',')+'){\n'

    for method_item in node.body:
        code_snippet+=func_dict[type(method_item).__name__](method_item)
    return code_snippet+'}\n'


In [9]:
def LocalVariableDeclaration2Code(node):
    code_snippet=Type2Code(node.type)+' '
    for declarator in node.declarators:
        code_snippet += func_dict[type(declarator).__name__](declarator)
    return code_snippet#+'; \n'

In [45]:
def VariableDeclarator2Code(node):
    code_snippet=node.name
    if node.initializer:
        code_snippet+= ' = '+func_dict[type(node.initializer).__name__](node.initializer)
    return code_snippet+';\n'

In [11]:
def VariableDeclaration2Code(node):
    code_snippet=''
    code_snippet+=Type2Code(node.type)+' '
    operandr=''
    operandl=''

    for declarator in node.declarators:
        operandr+=declarator.name +','
        operandl += func_dict[type(declarator.initializer).__name__](declarator.initializer)+','

    code_snippet +=operandr.rstrip(',')+' = '+operandl.rstrip(',')
    return code_snippet
    

In [12]:
def Type2Code(node):

    if type(node).__name__ == 'BasicType':
        return node.name +('[]'*len(node.dimensions) if node.dimensions else '')
    elif type(node).__name__ == 'ReferenceType':
        return node.name +('[]'*len(node.dimensions) if node.dimensions else '')
    
    

In [13]:
def ClassCreator2Code(node):# 미완
    
    argments='('
    for arg in node.arguments:
        argments+=func_dict[type(arg).__name__](arg)
    argments+=')'
    code_snippet='new '+node.type.name+argments
    return code_snippet

In [14]:
def MemberReference2Code(node):
    post_operator=[operator+' ' for operator in node.postfix_operators]
    code_snippet=' '.join(post_operator)+(node.qualifier+'.' if node.qualifier else '')+node.member
    selector=''
    for index in node.selectors:
        selector+=func_dict[type(index).__name__](index)
    code_snippet+=selector
    return code_snippet

In [15]:
def ForStatement2Code(node):
    code_snippet='for ( '
    control= node.control
    if node.control.init:
        code_snippet+= VariableDeclaration2Code(node.control.init)+';'
    if node.control.condition:
        code_snippet+=func_dict[type(node.control.condition).__name__](node.control.condition) +' ; '
    if node.control.update:
        for update in node.control.update:
            code_snippet+=func_dict[type(update).__name__](update)
    code_snippet+= ';)\n'
    
    code_snippet+= func_dict[type(node.body).__name__](node.body)
        
    return code_snippet

In [16]:
def IfStatement2Code(node):
    code_snippet='if ( '
    code_snippet+= func_dict[type(node.condition).__name__](node.condition)+')'
    code_snippet+='{\n'+func_dict[type(node.then_statement).__name__](node.then_statement)+'\n}\n'
    if node.else_statement:
        code_snippet+='else '+func_dict[type(node.else_statement).__name__](node.else_statement)
    return code_snippet
    
    

In [17]:
def BlockStatement2Code(node):
    code_snippet="{"
    for block_item in node.statements:
        code_snippet+=func_dict[type(block_item).__name__](block_item)
    code_snippet+="}"
    return code_snippet
    

In [18]:
def BinaryOperation2Code(node):
    code_snippet=func_dict[type(node.operandl).__name__](node.operandl)
    code_snippet+=' ' +node.operator +' '
    code_snippet+=func_dict[type(node.operandr).__name__](node.operandr)
    return code_snippet

In [19]:
def Literal2Code(node):
    
    return node.value

In [20]:
def ArrayCreator2Code(node):
    code_snippet='new '
    code_snippet+= Type2Code(node.type)
    for dim in node.dimensions:
        code_snippet+= '['+func_dict[type(dim).__name__](dim)+']'
    return code_snippet

In [21]:
def Cast2Code(node):
    code_snippet='('+Type2Code(node.type)+')'
    code_snippet+=func_dict[type(node.expression).__name__](node.expression)
    return code_snippet

In [22]:
def StatementExpression2Code(node):
    code_snippet=''
    # print(node)
    code_snippet+=func_dict[type(node.expression).__name__](node.expression)
    return code_snippet+';\n'

In [23]:
def Assignment2Code(node):
    code_snippet=func_dict[type(node.expressionl).__name__](node.expressionl)

    code_snippet+=' ' +node.type+' '
    code_snippet+= func_dict[type(node.value).__name__](node.value)
    return code_snippet

In [24]:
def MethodInvocation2Code(node):
    code_snippet=node.qualifier+('.' if node.qualifier else '')
    code_snippet+=node.member+'('
    args=''
    
    for arg in node.arguments:
        args+=func_dict[type(arg).__name__](arg)+','
    code_snippet+=args.rstrip(',')+')'
    return code_snippet#+';\n'

In [25]:
def ArraySelector2Code(node):
    code_snippet='['
    code_snippet+=func_dict[type(node.index).__name__](node.index)
    code_snippet+=']'
    return code_snippet

In [26]:
def ReturnStatement2Code(node):
    return 'return '+func_dict[type(node.expression).__name__](node.expression)

In [27]:
root.types[0].body[0].body[3].body.statements[1].expression.expressionl


NameError: name 'root' is not defined

In [36]:
root.types[0].body[2].body[0].then_statement.expression

MemberReference(member=a, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])

In [44]:
print(ClassDeclaration2Code(root.types[0]))


public  class codeforces{
static public void main(String[] args)Scanner sc = new Scanner(System.in); 
PrintWriter out = new PrintWriter(System.out); 
int test = 1; 
for ( int ind = 0; ind < test ; ++ ind)
{int[] a = new int[3]; 
;
a[0] = sc.nextInt();
a[1] = sc.nextInt();
a[2] = sc.nextInt();
Arrays.sort(a);
int k1 = a[0]; 
;
int k2 = a[1]; 
;
int k3 = a[2]; 
;
if ( k1 == 1 || k2 == 1 || k3 == 1){out.println("YES");
}else if ( k1 == 2 && k2 == 2 || k2 == 2 && k3 == 2){out.println("YES");
}else if ( k1 == 3 && k2 == 3 && k3 == 3){out.println("YES");
}else if ( k1 == 2 && k2 == 4 && k3 == 4){out.println("YES");
}else out.println("NO");
}out.flush()static void shuffle(int[] a)int n = a.length; 
for ( int i = 0; i < n ; ++ i)
{int r = i + (int)Math.random() * n - i; 
;
int tmp = a[i]; 
;
a[i] = a[r];
a[r] = tmp;
}static long gcd(long a,long b)if ( b == 0)return areturn .gcd(b,a % b)}


In [154]:
root.types[0].body[0].body[3].body.statements[8].else_statement.else_statement.else_statement.else_statement


StatementExpression(expression=MethodInvocation(arguments=[Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="NO")], member=println, postfix_operators=[], prefix_operators=[], qualifier=out, selectors=[], type_arguments=None), label=None)

In [88]:
a=[]
for i in a:
    print('?')

In [167]:

print(ForStatement2Code(root.types[0].body[0].body[3]))

for ( int ind = 0; ind < test ; ++ ind)
{int[] a = new int[3];
a[0] = sc.nextInt();
a[1] = sc.nextInt();
a[2] = sc.nextInt();
Arrays.sort(a);
int k1 = a[0];
int k2 = a[1];
int k3 = a[2];
if ( k1 == 1 || k2 == 1 || k3 == 1){out.println("YES");
}else if ( k1 == 2 && k2 == 2 || k2 == 2 && k3 == 2){out.println("YES");
}else if ( k1 == 3 && k2 == 3 && k3 == 3){out.println("YES");
}else if ( k1 == 2 && k2 == 4 && k3 == 4){out.println("YES");
}else out.println("NO");
}


In [33]:
ArrayCreator

NameError: name 'ArrayCreator' is not defined

In [47]:
str(type(tree))

"<class 'javalang.tree.CompilationUnit'>"

In [176]:
a='[]'
a[:-1]

'['

In [156]:
te=''
for i in range(8):
    te+=str(i)+','

In [159]:
te.rstrip(',')

'0,1,2,3,4,5,6,7'

In [160]:
a=''
a.rstrip(',')

''